In [ ]:
# Import findspark and initialize.
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',')
df.createOrReplaceTempView('home_sales')



In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = '''
SELECT
    YEAR(date) AS year,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    YEAR(date)
ORDER BY
    year
'''

# Execute the query and show the results
spark.sql(query).show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = '''
SELECT
    date_built AS year_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3
GROUP BY
    date_built
ORDER BY
    year_built
'''

# Execute the query and show the results
spark.sql(query).show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = '''
SELECT
    date_built AS year_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3
    AND bathrooms = 3
    AND floors = 2
    AND sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built
'''

# Execute the query
spark.sql(query).show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# SQL query to get the average price per "view" rating with price >= 350,000
query = '''
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
'''

# Execute the query
result = spark.sql(query)

# Show the result
result.show()

# End time to calculate the query run time
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.9832239151000977 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

# Start the timer for cached query execution
start_time_cached = time.time()

# Run the query on the cached data
query_cached = '''
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
GROUP BY
    view
HAVING
    avg_price >= 350000
ORDER BY
    view DESC
'''

# Execute the cached query
spark.sql(query_cached).show()

# Calculate and print the runtime for cached query
print("Cached query runtime: --- %s seconds ---" % (time.time() - start_time_cached))

# Now, let's run the same query on uncached data and compare the times.
# Start the timer for uncached query execution
start_time_uncached = time.time()

# Run the query on the uncached data
df.createOrReplaceTempView('home_sales')  # Recreate the temporary view with uncached data

# Execute the uncached query
spark.sql(query_cached).show()

# Calculate and print the runtime for uncached query
print("Uncached query runtime: --- %s seconds ---" % (time.time() - start_time_uncached))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Cached query runtime: --- 0.9673936367034912 seconds ---
+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Uncached query runtime: ---

In [ ]:
# Use mode("overwrite") to overwrite the existing files in the path
df.write.mode("overwrite").parquet("folder")




In [ ]:
# 11. Read the formatted parquet data.
df_parquet = spark.read.parquet("folder")



In [ ]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("parquet_table")


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.


# Define the SQL query to calculate average price per view rating
query = '''
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    parquet_table
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
'''


start_time_parquet = time.time()

result_parquet = spark.sql(query)

result_parquet.show()

end_time_parquet = time.time()
print("Parquet query runtime: --- %s seconds ---" % (end_time_parquet - start_time_parquet))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Parquet query runtime: --- 1.3739087581634521 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
print("home_sales cached:", spark.catalog.isCached("home_sales"))


home_sales cached: False
